In [5]:
from PIL import Image
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
from pathlib import Path

def create_chart(csv_filename: str,
                date_column: str,
                chart_type: str,
                columns_to_plot: dict,
                data_folder: str = "data",
                watermark: str = "off",
                title: str = None,
                stacked: str = "off",
                unit: str = None,
                columns_to_plot_right: dict = None,
                unit_right: str = None,
                ticker_space: float = 1000000000,
                ticker_space_right: float = 0.05,
                chart_type_right: str = "line") -> go.Figure:

    # Try different path options
    possible_paths = [
        Path.cwd() / data_folder / csv_filename,
        Path.cwd().parent / data_folder / csv_filename,
        Path(data_folder) / csv_filename
    ]

    # Print current working directory for debugging
    print(f"Current working directory: {Path.cwd()}")

    # Try each path until we find one that works
    df = None
    for path in possible_paths:
        try:
            if path.exists():
                df = pd.read_csv(path)
                print(f"Successfully loaded data from: {path}")
                break
        except Exception as e:
            print(f"Tried path: {path} (not found)")
            continue

    if df is None:
        raise FileNotFoundError(f"Could not find {csv_filename} in any of these locations:\n" +
                                "\n".join(str(p) for p in possible_paths))

    # Convert date column to datetime
    df[date_column] = pd.to_datetime(df[date_column])

    # Create figure
    fig = go.Figure()

    # Generate hover template text based on unit
    def get_hover_template(display_name, is_right_axis=False):
        current_unit = unit_right if is_right_axis else unit
        if current_unit is None:
            return f"{display_name}: %{{y:.2f}}<extra></extra>"
        elif current_unit.upper() == 'K':
            return f"{display_name}: %{{y:,.0f}}K<extra></extra>"
        elif current_unit.upper() == 'M':
            return f"{display_name}: %{{y:,.1f}}M<extra></extra>"
        elif current_unit.upper() == 'B':
            return f"{display_name}: %{{y:,.2f}}B<extra></extra>"
        return f"{display_name}: %{{y:.2f}}<extra></extra>"

    # Function to format tick values
    def tick_format(value, is_right_axis=False):
        try:
            val = float(value)
            current_unit = unit_right if is_right_axis else unit
            if current_unit is None:
                return f"{val:.2f}"
            elif current_unit.upper() == 'K':
                return f"{val/1000:,.0f}K"
            elif current_unit.upper() == 'M':
                return f"{val/1_000_000:,.1f}M"
            elif current_unit.upper() == 'B':
                return f"{val/1_000_000_000:,.2f}B"
            return f"{val:.2f}"
        except:
            return value

    # Function to add traces
    def add_trace(column, display_name, is_right_axis=False):
        yaxis = 'y2' if is_right_axis else 'y'
        # Set line color to black for right y-axis trace
        line_color = 'black' if is_right_axis else None
        current_chart_type = chart_type_right if is_right_axis else chart_type

        if current_chart_type.lower() == 'line':
            fig.add_trace(
                go.Scatter(
                    x=df[date_column],
                    y=df[column],
                    name=display_name,
                    mode='lines',
                    line=dict(color=line_color) if line_color else dict(),
                    hovertemplate=get_hover_template(display_name, is_right_axis),
                    yaxis=yaxis
                )
            )
        elif current_chart_type.lower() == 'bar':
            fig.add_trace(
                go.Bar(
                    x=df[date_column],
                    y=df[column],
                    name=display_name,
                    marker_color=line_color if line_color else None,
                    hovertemplate=get_hover_template(display_name, is_right_axis),
                    yaxis=yaxis
                )
            )
        elif current_chart_type.lower() == 'area':
            fig.add_trace(
                go.Scatter(
                    x=df[date_column],
                    y=df[column],
                    name=display_name,
                    mode='lines',
                    line=dict(color=line_color) if line_color else dict(),
                    stackgroup='one' if stacked.lower() == 'on' else None,
                    fill='tonexty',
                    hovertemplate=get_hover_template(display_name, is_right_axis),
                    yaxis=yaxis
                )
            )

    # Add traces for left y-axis
    for column, display_name in columns_to_plot.items():
        add_trace(column, display_name, is_right_axis=False)

    # Add traces for right y-axis if provided
    if columns_to_plot_right:
        for column, display_name in columns_to_plot_right.items():
            add_trace(column, display_name, is_right_axis=True)

    # Get min and max values for both y-axes
    def get_axis_range(columns_dict, current_chart_type):
        if stacked.lower() == 'on' and current_chart_type.lower() in ['bar', 'area']:
            stacked_sums = df[list(columns_dict.keys())].sum(axis=1)
            return 0, stacked_sums.max()
        else:
            y_min = min(df[column].min() for column in columns_dict.keys())
            y_max = max(df[column].max() for column in columns_dict.keys())
            return y_min, y_max

    y_min, y_max = get_axis_range(columns_to_plot, chart_type)
    if columns_to_plot_right:
        y2_min, y2_max = get_axis_range(columns_to_plot_right, chart_type_right)

    # Generate tick values with custom spacing
    def generate_ticks(y_min, y_max, spacing):
        tick_min = np.floor(y_min / spacing) * spacing
        tick_max = np.ceil(y_max / spacing) * spacing
        ticks = np.arange(tick_min, tick_max + spacing, spacing)
        return ticks[(ticks >= y_min - spacing) & (ticks <= y_max + spacing)]

    # Generate tick values and their formatted labels for both axes
    tick_vals = generate_ticks(y_min, y_max, ticker_space)
    tick_texts = [tick_format(val, False) for val in tick_vals]

    if columns_to_plot_right:
        tick_vals_right = generate_ticks(y2_min, y2_max, ticker_space_right)
        tick_texts_right = [tick_format(val, True) for val in tick_vals_right]
    
    # Calculate font sizes
    axis_size = 17

    # Define colors - darker grey
    border_color = '#7f7f7f'

    # Set default title text if not provided
    default_title = f'{chart_type.capitalize()} Chart: {", ".join(columns_to_plot.values())}'
    if columns_to_plot_right:
        default_title += f' vs {", ".join(columns_to_plot_right.values())}'
    if stacked.lower() == 'on' and chart_type.lower() in ['bar', 'area']:
        default_title = f'Stacked {default_title}'

    # Basic layout update with new styling
    layout_dict = {
        'title': dict(
            text=title if title else default_title,
            font=dict(
                color='#000000',
                size=29,
                weight='bold'
            ),
            x=0.055,
            y=0.94,
            xanchor='left',
            yanchor='top',
            pad=dict(t=0, b=0)
        ),
        'width': 1350,
        'height': 750,
        'showlegend': True,
        'font': dict(color=border_color),
        'legend': dict(
            orientation="h",
            yanchor="top",
            y=0.98,
            xanchor="left",
            x=0.02,
            font=dict(
                color=border_color,
                size=axis_size
            ),
            bgcolor='rgba(255, 255, 255, 0.8)'
        ),
        'xaxis': dict(
            title=None,
            tickformat="%b '%y",
            dtick="M3",
            tickangle=0,
            tickmode='array',
            ticktext=[
                date.strftime("%b '%y")
                for date in pd.date_range(
                    start=df[date_column].min(),
                    end=df[date_column].max(),
                    freq='3ME'
                )
            ],
            tickvals=[
                date
                for date in pd.date_range(
                    start=df[date_column].min(),
                    end=df[date_column].max(),
                    freq='3ME'
                )
            ],
            tickfont=dict(
                color=border_color,
                size=axis_size
            ),
            showgrid=False,
            ticks='outside',
            ticklen=8,
            tickwidth=1
        ),
        'yaxis': dict(
            title=None,
            showgrid=False,
            tickfont=dict(
                color=border_color,
                size=axis_size
            ),
            ticks='outside',
            ticklen=8,
            tickwidth=1,
            range=[min(tick_vals), max(tick_vals)],
            mirror=True,
            side='left',
            showticksuffix='none',
            showtickprefix='none',
            showticklabels=True,
            tickson='labels',
            tickmode='array',
            ticktext=tick_texts,
            tickvals=tick_vals
        )
    }

    # Add right y-axis if needed
    if columns_to_plot_right:
        layout_dict['yaxis2'] = dict(
            title=None,
            showgrid=False,
            tickfont=dict(
                color=border_color,
                size=axis_size
            ),
            ticks='outside',
            ticklen=8,
            tickwidth=1,
            range=[y2_min, y2_max],
            mirror=True,
            side='right',
            overlaying='y',
            showticksuffix='none',
            showtickprefix='none',
            showticklabels=True,
            tickson='labels',
            tickmode='array',
            ticktext=tick_texts_right,
            tickvals=tick_vals_right
        )

    # Add barmode for stacked bar charts
    if chart_type.lower() == 'bar' and stacked.lower() == 'on':
        layout_dict['barmode'] = 'stack'

    fig.update_layout(layout_dict)

    # Add watermark and styling if requested
    if watermark.lower() == "on":
        try:
            # Open and add the watermark image
            watermark_img = Image.open("glassnode_large.png")
            fig.add_layout_image(
                dict(
                    source=watermark_img,
                    sizex=0.36,
                    sizey=0.36,
                    xanchor="center",
                    yanchor="middle",
                    xref="paper",
                    yref="paper",
                    x=0.5,
                    y=0.5,
                    opacity=0.15,
                    layer="above"
                )
            )

            # Add copyright text
            fig.add_annotation(
                showarrow=False,
                text=f"© {str(datetime.today().year)} Glassnode. All Rights Reserved",
                font=dict(
                    size=15,
                    color=border_color
                ),
                xref='paper',
                yref='paper',
                x=1,
                y=-0.125,
                opacity=0.5
            )
        except FileNotFoundError:
            print("Warning: Watermark image not found. Skipping watermark but applying styling.")

        # Apply styling regardless of whether watermark image was found
        fig.update_layout({
            'plot_bgcolor': 'white',
            'paper_bgcolor': 'white',
            'xaxis': {
                'linecolor': border_color,
                'linewidth': 1,
                'mirror': True,
                'showgrid': False,
                'ticks': 'outside',
                'ticklen': 8,
                'tickwidth': 1
            },
            'yaxis': {
                'showgrid': False,
                'linecolor': border_color,
                'linewidth': 1,
                'mirror': True,
                'ticks': 'outside',
                'ticklen': 8,
                'tickwidth': 1,
                'range': [min(tick_vals), max(tick_vals)],
                'side': 'left',
                'showticksuffix': 'none',
                'showtickprefix': 'none',
                'showticklabels': True,
                'tickson': 'labels'
            },
            'yaxis2': {
                'showgrid': False,
                'linecolor': border_color,
                'linewidth': 1,
                'mirror': True,
                'ticks': 'outside',
                'ticklen': 8,
                'tickwidth': 1,
                'range': [y2_min, y2_max] if columns_to_plot_right else None,
                'side': 'right',
                'showticksuffix': 'none',
                'showtickprefix': 'none',
                'showticklabels': True,
                'tickson': 'labels'
            } if columns_to_plot_right else {},
            'autosize': True
        })

    return fig

In [6]:

# Example usage with dual y-axes
fig_area = create_chart(
        csv_filename='mstr_data.csv',
        date_column='t',
        chart_type='area',
        columns_to_plot={
            'BalanceBTC': 'BTC Holding [BTC]'
        },
        columns_to_plot_right={
            'Close': 'MSTR Stock Price [USD]'
        },
        watermark="on",
        title='Microstrategy BTC Holding [BTC]',
        stacked="off",
        unit='K',  # Will show actual numbers with 2 decimal places for left axis
        unit_right='None',  # Will show numbers in thousands for right axis
        ticker_space=10000000000,  # 2B spacing for left axis
        ticker_space_right=0.05   # 0.1 spacing for right axis
    )
fig_area.show()

Current working directory: /Users/cryptovizart/Desktop/glassnode/glassnode_chart/fasanara
Successfully loaded data from: /Users/cryptovizart/Desktop/glassnode/glassnode_chart/data/mstr_data.csv
